In [ ]:
import numpy as np
import scipy.io as io
import pandas as pd
import sys
import os

In [ ]:
directory = '/home/data/Project_ASD/' + os.environ["USER"]+'/' #feel free to upload data from /Project_ASD/seyda
data = np.load(directory+'targetP3s_equal.npz', allow_pickle=True)
targetP3s_equal = data['targetP3s_equal']
targetP3s_equal = data['targetP3s_equal']
subjects = data['subjects']
n_subj,n_elec,n_trials=targetP3s_equal.shape


In [ ]:
#save a moving average figure for each subject 

fig_dir = directory+ '/single-trial-figures/'
for g,s,subject in subjects:
    P3s=targetP3s[int(s),0]
    conds=targetP3s[int(s),1]

    subject=subject[:-4]
    GFP=np.std(P3s, axis=0)
    winSize=30
    y=moving_average(GFP, n=winSize)

    dotSize=len(y)/300
    plt.close()
    plt.rcParams["figure.figsize"] = [20,10]

    plt.scatter(range(len(y)),y,dotSize)

    yC=moving_average(conds, n=winSize)
    plt.scatter(range(len(yC)),yC,dotSize)
    plt.title(subject)
    plt.ylim(0,7)
    plt.savefig(fig_dir+'P3-'+subject+'.png')
    


In [ ]:
#In the first two sessions, we have: C 1 2 3 4 1. Say 1a 2 3 4 1b
indC12341=np.empty((subjects.shape[0],6),int)
ind_exclude_subjects=[]
for s in range(subjects.shape[0]):
    #P3s=targetP3s[int(s),0]
    conds=targetP3s[s,1]
    ind_C1a=np.where(conds == 1)[0][0]
    ind_C2=np.where(conds == 2)[0][0]
    ind_C3=np.where(conds == 3)[0][0]
    ind_C4=np.where(conds == 4)[0][0]
    ind_C1b=ind_C4+np.where(conds[ind_C4:] == 1)[0][0]
    ind_ses3=ind_C1b+np.where(conds[ind_C1b:]!= 1)[0][0] #index of where ses3 starts in target array 
    indC12341[s,]=[ind_C1a,ind_C2,ind_C3,ind_C4,ind_C1b,ind_ses3]
    #print(subject,ind_C1a,ind_C2,ind_C3,ind_C4,ind_C1b,ind_ses3)
    try:
        assert ind_C1a<ind_C2<ind_C3<ind_C4<ind_C1b<ind_ses3
    except AssertionError:
        ind_exclude_subjects.append(int(s))
print('subject indices to exclude based on design of session 1 and 2: '+str(ind_exclude_subjects))



In [ ]:
#select eequal number of trials for each 
num_trials=np.round((np.mean(np.diff(indC12341),axis=0)-np.std(np.diff(indC12341),axis=0)),0)
num_trials

In [ ]:
#select eequal number of trials for each 

targetP3s_equal=np.empty((subjects.shape[0],160,int(sum(num_trials))),dtype=float) #(41, 160, 380)
for s in range(subjects.shape[0]):
    #for C1, donnt include the first trials they are just training 
    interval=np.arange(20,indC12341[s,1],1)
    if len(interval)>num_trials[0]:
        inds=sorted(np.random.choice(interval,int(num_trials[0]), replace=False))
    else:
        dfrnc=int(num_trials[0]-len(interval))
        inds=sorted(np.append(interval,np.random.choice(interval,dfrnc, replace=False)))

    #for other conditions
    for c in range(1,5):
        interval=np.arange(indC12341[s,c],indC12341[s,c+1],1)
        if len(interval)>num_trials[c]:
            ind=sorted(np.random.choice(interval,int(num_trials[c]), replace=False))
        else:
            dfrnc=int(num_trials[c]-len(interval))
            ind=sorted(np.append(interval,np.random.choice(interval,dfrnc, replace=True)))
        inds=np.append(inds,ind)
    targetP3s_equal[s,]=targetP3s[s,0][:,inds]


    #argetP3s_equal[s,0:num_trials[1]]=targetP3s[s,c]
    #nd
    #arr=np.arange(indC12341[s,c],indC12341[s,c+1],1)
    #ind=sorted(np.random.choice(arr,int(num_trials[c]), replace=False) )
    #targetP3s_equal[s,]=

In [ ]:
#Save your results 
directory = '/home/data/Project_ASD/' + os.environ["USER"]+'/' #feel free to upload data from /Project_ASD/seyda
np.savez(directory+'targetP3s_equal.npz', targetP3s_equal=targetP3s_equal, subjects=subjects)


In [ ]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [ ]:
data = np.load(directory+'targetP3s_equal.npz', allow_pickle=True)
targetP3s_equal = data['targetP3s_equal']
subjects = data['subjects']
n_subj,n_elec,n_trials=targetP3s_equal.shape

In [ ]:
#remove outlier subjects
o=[]
for t in range(n_trials):
    for e in range(n_subj):
        y=targetP3s_equal[:,e,t]
        o.append(find_outliers(y))
#print(sorted(o))
out_subj=[4,31]
targetP3s_equal=np.delete(targetP3s_equal, out_subj, axis=0)
subjects=np.delete(subjects, out_subj, axis=0)

#nans, x= nan_helper(y)
#y[nans]= np.interp(x(nans), x(~nans), y[~nans])
#targetP3s_equal[s,e]=y


In [ ]:
#take group means
ind_asd=np.where(subjects[:,0]=='0')
ind_nt=np.where(subjects[:,0]=='1')

u_asd=np.mean(targetP3s_equal[ind_asd],axis=0)
u_nt=np.mean(targetP3s_equal[ind_nt],axis=0)

#u_asd_gfp=u_asd[19]
#u_nt_gfp=u_nt[19]
u_asd_gfp=np.std(u_asd, axis=0)
u_nt_gfp=np.std(u_nt, axis=0)

u_asd_gfp[find_outliers(u_asd_gfp)]=np.nan
u_nt_gfp[find_outliers(u_nt_gfp)]=np.nan

nans, x= nan_helper(u_asd_gfp)
u_asd_gfp[nans]= np.interp(x(nans), x(~nans), u_asd_gfp[~nans])

nans, x= nan_helper(u_nt_gfp)
u_nt_gfp[nans]= np.interp(x(nans), x(~nans), u_nt_gfp[~nans])



In [ ]:
#calculate conditions to plot 
num_trials=num_trials.astype(int)
n1a,n2,n3,n4,n1b=num_trials
conds=np.concatenate((np.zeros(n1a)+1,np.zeros(n2)+2,np.zeros(n3)+3,np.zeros(n4)+4,np.zeros(n1b)+1),axis=0)

In [ ]:

winSize=30
y_asd=moving_average(u_asd_gfp, n=winSize)
y_nt=moving_average(u_nt_gfp, n=winSize)

dotSize=len(y_asd)/30
plt.close()
plt.rcParams["figure.figsize"] = [20,10]

plt.scatter(range(len(y_nt )),y_nt ,dotSize)
plt.scatter(range(len(y_asd)),y_asd,dotSize)
plt.scatter(range(len(conds)),conds/2,dotSize)
title='Target P3 GFP amplitudes of single trials during Session 1 & 2 averaged for groups'
#title='Target P3 amplitudes of single trials at Ch19 during Session 1 & 2 averaged for groups'
plt.title(title)
plt.savefig(fig_dir+'P3-'+title+'.png')

#plt.ylim(0,14)


In [ ]:
from scipy.signal import find_peaks
for s in range(subjects.shape[0]):
    peaks, _ = find_peaks(x, prominence=1)      # BEST!
    x=targetP3s_equal[s]
    x=np.std(x,axis=0)
    x=moving_average(x, n=10)

    plt.plot(peaks, x[peaks], "ob"); plt.plot(x); plt.legend(['prominence'])

plt.show()